## **Compute risk bound and theta with Selection with Guaranteed Risk (SGR)**

Reference: https://arxiv.org/pdf/1705.08500 Algorithm 1

In [4]:
import risk_control

In [ ]:
risk_stars = [0.02, 0.1, 0.15, 0.20, 0.25]
delta = 0.001 ## confidence